# VacationPy
---

## Import Libraries and Load the Weather and Coordinates Data

In [35]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from scipy.constants import convert_temperature

# Import API key
from api_keys import geoapify_key

In [36]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Change Temperature to Celsius
city_data_df["Max Temp"] = convert_temperature(city_data_df["Max Temp"], old_scale="Kelvin", new_scale="Celsius")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,5.23,59,41,2.68,US,1731132422
1,1,blackmans bay,-43.0167,147.3167,16.07,52,100,3.39,AU,1731132547
2,2,port alfred,-33.5906,26.8910,20.66,86,58,1.79,ZA,1731132549
3,3,bodo,67.2800,14.4050,6.54,74,40,7.20,NO,1731132550
4,4,bethel,41.3712,-73.4140,8.62,50,0,5.66,US,1731132551


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [37]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
     "Lng",
     "Lat",
     geo=True,
     size= "Humidity",
     scale= 0.75,
     color= "City",
     tiles = "OSM",
     frame_height=600,
     frame_width= 800)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [38]:
# Narrow down cities that fit criteria and drop any results with null values
filt_city_df = city_data_df.loc[(city_data_df["Cloudiness"] <= 10) & (city_data_df["Wind Speed"] <= 4.5) & (city_data_df["Max Temp"] <= 27) & (city_data_df["Max Temp"] >= 21)]

# Drop any rows with null values
clean_filt_df = filt_city_df.dropna(how="any")

# Display sample data
clean_filt_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,cabo san lucas,22.8909,-109.9124,22.09,56,0,1.47,MX,1731132575
73,73,tazacorte,28.6290,-17.9293,25.64,31,0,4.02,ES,1731132630
80,80,tangi,34.3006,71.6536,26.39,34,0,1.07,PK,1731132640
306,306,churachandpur,24.3333,93.6667,21.78,62,7,3.13,IN,1731132890
426,426,gunjur,13.2019,-16.7339,24.04,94,2,2.57,GM,1731133029
456,456,urangan,-25.3000,152.9000,25.07,77,1,2.57,AU,1731133062
480,480,bonthe,7.5264,-12.5050,24.50,90,2,1.69,SL,1731133088
493,493,mulanje,-16.0316,35.5000,25.95,61,5,0.68,MW,1731133104


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_filt_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
25,cabo san lucas,MX,22.8909,-109.9124,56,
73,tazacorte,ES,28.6290,-17.9293,31,
80,tangi,PK,34.3006,71.6536,34,
306,churachandpur,IN,24.3333,93.6667,62,
426,gunjur,GM,13.2019,-16.7339,94,
456,urangan,AU,-25.3000,152.9000,77,
480,bonthe,SL,7.5264,-12.5050,90,
493,mulanje,MW,-16.0316,35.5000,61,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":1,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
tazacorte - nearest hotel: App Leyma
tangi - nearest hotel: No hotel found
churachandpur - nearest hotel: FSW IB.Road
gunjur - nearest hotel: Footsteps Eco Lodge
urangan - nearest hotel: Ramada
bonthe - nearest hotel: No hotel found
mulanje - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
25,cabo san lucas,MX,22.8909,-109.9124,56,Comfort Rooms
73,tazacorte,ES,28.6290,-17.9293,31,App Leyma
80,tangi,PK,34.3006,71.6536,34,No hotel found
306,churachandpur,IN,24.3333,93.6667,62,FSW IB.Road
426,gunjur,GM,13.2019,-16.7339,94,Footsteps Eco Lodge
456,urangan,AU,-25.3000,152.9000,77,Ramada
480,bonthe,SL,7.5264,-12.5050,90,No hotel found
493,mulanje,MW,-16.0316,35.5000,61,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
     "Lng",
     "Lat",
     geo=True,
     size= "Humidity",
     scale= 0.75,
     color= "City",
     tiles = "OSM",
     frame_height= 600,
     frame_width= 800,
     hover_cols= ["Hotel Name", "Country"])

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)